# Milestone 2 Inference

Nama : Dwi Putra Satria Utama

Batch : 20

# Import libraries

In [1]:
# Library Pengolahan Data
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model


# Load model

In [2]:
# load model
model_inf = load_model('best_model')

# Text prepocessing

In [3]:
# Load the dictionary slang/abbreviations menjadi kalimat standar (baku)
slang_to_standard = {}
with open('chatwords.txt', 'r') as file:
    for line in file:
        words = line.strip().split()
        slang = words[0].lower()
        standard = ' '.join(words[1:]).lower()
        slang_to_standard[slang] = standard

stop_words = set(stopwords.words('english'))

stemmer = PorterStemmer()
lem = WordNetLemmatizer()

def is_alpha(word):
    for part in word.split('-'):
        if not part.isalpha():
            return False
    return True

def tweets_proses(tweets):
    # Mengubah Teks ke Lowercase
    tweets = tweets.lower()

    # Menghilangkan Mention
    tweets = re.sub("@[A-Za-z0-9_]+", " ", tweets)

    # Menghilangkan Hashtag
    tweets = re.sub("#[A-Za-z0-9_]+", " ", tweets)

    # Menghilangkan \n
    tweets = re.sub(r"\\n", " ",tweets)

    # Menghilangkan Whitespace
    tweets = tweets.strip()

    # Menghilangkan Link
    tweets = re.sub(r"http\S+", " ", tweets)
    tweets = re.sub(r"www.\S+", " ", tweets)

    # Menghilangkan yang Bukan Huruf seperti Emoji, Simbol Matematika (seperti μ), dst
    tweets = re.sub("[^A-Za-z\s']", " ", tweets)

    # Menghilangkan RT
    tweets = re.sub("rt", " ",tweets)

    # Melakukan Tokenisasi
    tokens = word_tokenize(tweets)

    # Memecah teks menjadi token (kata-kata) menggunakan word_tokenize dan menyimpan kata-kata yang hanya berisi huruf tanda _
    words = [word for word in word_tokenize(tweets) if is_alpha(word)]

    # Melakukan lemmatisasi pada setiap kata untuk mengubahnya ke bentuk kata dasar
    words = [lem.lemmatize(word) for word in words]

    # Membuang kata-kata yang termasuk dalam stopwords (kata-kata umum yang sering tidak memberikan banyak informasi).
    words = [w for w in words if not w in stop_words]

    # Mengganti slang and abbreviations dengan kalimat baku
    words = [slang_to_standard[word.lower()] if word.lower() in slang_to_standard else word for word in words]

    # Stemming menggunakan NLTK Porter Stemmer
    #words = [stemmer.stem(word) for word in words]

    # Joining the words back to form the processed text
    text = " ".join(words)



    return text


# Create new data

In [4]:
# Create New Data 

data_inf = {
    'tweets' : 'Just tried ChatGPT and it is mind-blowing! The AI model can generate human-like responses and engage in meaningful conversations. Impressive work by OpenAI'                                
}

data_inf = pd.DataFrame([data_inf])
data_inf

,tweets
0,Just tried ChatGPT and it is mind-blowing! The...


# Proses data new data

In [5]:
# Preprocessing Data Inference
data_inf['tweets_processed'] = data_inf['tweets'].apply(lambda x: tweets_proses(x))

# Predict

In [6]:
# Prediksi jenis tweet
y_inf_pred = np.argmax(model_inf.predict(data_inf['tweets_processed']), axis=-1)

# Membuat fungsi untuk return result prediksi
if y_inf_pred[0] == 0:
    result = 'bad'
elif y_inf_pred[0] == 1:
    result = 'good'
else:
    result = 'neutral'

# Print Result
print(result)


1/1 [==============================] - 1s 1s/step
good


Berdasarkan hasil tersebut bahwa kalimat atau tweets tersebut termasuk good sentimen